In [1]:
# === Binary-augmented supernova cosmology test — one-shot Colab cell ===
# Installs, builds Fibonacci constants, creates model families, fits everything,
# prints fair metrics (reduced χ², AIC/BIC), and saves plots+CSV to /content/out.

# 0) Quiet installs
import sys, subprocess, importlib
def _pip(pkg): subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
for pkg in ["numpy", "pandas", "matplotlib", "scipy", "requests"]:
    try: importlib.import_module(pkg)
    except Exception: _pip(pkg)

# 1) Imports
import os, io, math, warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Dict, Tuple, Optional, List
warnings.filterwarnings("ignore")
from scipy.optimize import minimize

# --------------------------- Utilities ---------------------------

def http_get(url: str, timeout: int = 30) -> Optional[bytes]:
    import requests
    try:
        r = requests.get(url, timeout=timeout)
        return r.content if r.status_code == 200 else None
    except Exception:
        return None

def safe_logdet(C: np.ndarray) -> float:
    sign, logdet = np.linalg.slogdet(C)
    if not np.isfinite(logdet) or sign <= 0:
        eps = 1e-12 * float(np.mean(np.diag(C)))
        C = C + eps * np.eye(C.shape[0])
        sign, logdet = np.linalg.slogdet(C)
    return float(logdet)

def analytic_delta_chi2(mu_obs: np.ndarray, mu_model: np.ndarray, Cinv: np.ndarray) -> Tuple[float, float]:
    """Best-fit intercept Δ and minimal chi^2 for residuals r = mu_obs - (mu_model + Δ)."""
    r = mu_obs - mu_model
    u = np.ones_like(r)
    A = float(u @ (Cinv @ u))
    B = float(u @ (Cinv @ r))
    delta_star = B / A
    chi2_min = float(r @ (Cinv @ r) - (B**2)/A)
    return chi2_min, delta_star

def r_squared(y: np.ndarray, yhat: np.ndarray) -> float:
    ss_res = float(np.sum((y - yhat)**2))
    ss_tot = float(np.sum((y - np.mean(y))**2))
    return 1.0 - ss_res/ss_tot

def aic_bic(chi2_min: float, C: np.ndarray, k_params: int) -> Tuple[float, float, float]:
    N = C.shape[0]
    logdet = safe_logdet(C)
    logL = -0.5 * (chi2_min + logdet + N * math.log(2*math.pi))
    AIC = 2*k_params - 2*logL
    BIC = k_params*math.log(N) - 2*logL
    return float(logL), float(AIC), float(BIC)

# --------------------------- Fibonacci word & constants ---------------------------

def fib_bits(N: int) -> str:
    """First N bits of the Fibonacci word (0->01, 1->0)."""
    s = "0"
    while len(s) < N:
        s = "".join("01" if ch == "0" else "0" for ch in s)
    return s[:N]

def complement_bits(bits: str) -> str:
    return "".join("1" if b == "0" else "0" for b in bits)

def decimal_digits_value(bits: str) -> float:
    """Interpret bits as decimal digits: 0.b1 b2 ... in base 10."""
    return int(bits, 10) / (10**len(bits)) if False else (int(bits) / (10**len(bits)))

def binary_fraction_value(bits: str) -> float:
    """Interpret bits as a binary fraction: sum b_i 2^{-i} (float)."""
    # stable sum from the right
    acc = 0.0
    for b in reversed(bits):
        acc = (acc + (1.0 if b == "1" else 0.0)) * 0.5
    return acc

def make_binary_constants(n_digits: int = 200, n_bin_bits: int = 200) -> Dict[str, float]:
    F = fib_bits(max(n_digits, n_bin_bits))
    Fd, Fb = F[:n_digits], F[:n_bin_bits]
    R = complement_bits(F)
    Rd, Rb = R[:n_digits], R[:n_bin_bits]

    DF = decimal_digits_value(Fd)                # ~0.0100101...
    DR = decimal_digits_value(Rd)                # complement as digits
    BF = binary_fraction_value(Fb)               # in [0,1]
    BR = 1.0 - BF                                # exact complement in infinite limit

    consts = {
        "DF": DF,
        "DR": DR,
        "OneMinusDF": 1.0 - DF,
        "BinF": BF,
        "BinR": BR,
        "neg_DF": -DF,
        "neg_DR": -DR,
        "neg_OneMinusDF": -(1.0 - DF),
        "neg_BinF": -BF,
        "neg_BinR": -BR,
    }
    return consts

# --------------------------- Cosmology models ---------------------------

@dataclass
class CosmoModel:
    name: str
    fit_omegam: bool
    params: Dict[str, float] = field(default_factory=dict)
    def predict_mu(self, z: np.ndarray, H0: float = 70.0) -> np.ndarray:
        raise NotImplementedError

def E2_LCDM(z: np.ndarray, Om: float) -> np.ndarray:
    return Om*(1+z)**3 + (1-Om)

def E2_constw(z: np.ndarray, Om: float, w0: float) -> np.ndarray:
    return Om*(1+z)**3 + (1-Om)*(1+z)**(3*(1+w0))

def E2_w0wa(z: np.ndarray, Om: float, w0: float, wa: float) -> np.ndarray:
    g = (1+z)**(3*(1+w0+wa)) * np.exp(-3*wa*z/(1+z))
    return Om*(1+z)**3 + (1-Om)*g

def E2_binary(z: np.ndarray, Om: float) -> np.ndarray:
    ln1pz = np.log(1+z)
    g = (1+z)**3 * np.exp((3.0/(2.0*math.log(2.0)))*(ln1pz**2))
    return Om*(1+z)**3 + (1-Om)*g

def E2_scale_log(z: np.ndarray, Om: float, c: float) -> np.ndarray:
    """w(a) = -c log2 a  =>  ρ_de ∝ (1+z)^3 * exp( (3c/(2 ln 2)) [ln(1+z)]^2 )."""
    ln1pz = np.log(1+z)
    g = (1+z)**3 * np.exp((3.0*c/(2.0*math.log(2.0)))*(ln1pz**2))
    return Om*(1+z)**3 + (1-Om)*g

def E2_offset_log(z: np.ndarray, Om: float, b: float) -> np.ndarray:
    """w(a) = -log2 a - b  =>  ρ_de ∝ (1+z)^{3(1-b)} * exp( (3/(2 ln 2)) [ln(1+z)]^2 )."""
    ln1pz = np.log(1+z)
    g = (1+z)**(3*(1.0-b)) * np.exp((3.0/(2.0*math.log(2.0)))*(ln1pz**2))
    return Om*(1+z)**3 + (1-Om)*g

def distance_modulus(z: np.ndarray, E2_func, H0: float, Om: float, **kwargs) -> np.ndarray:
    zmax = float(np.max(z))
    z_grid = np.linspace(0.0, max(zmax, 1e-6), 4000)

    if E2_func is E2_constw:
        w0 = kwargs.get("w0", -1.0); E2_grid = E2_constw(z_grid, Om, w0)
    elif E2_func is E2_w0wa:
        w0 = kwargs.get("w0", -1.0); wa = kwargs.get("wa", 0.0); E2_grid = E2_w0wa(z_grid, Om, w0, wa)
    elif E2_func is E2_binary:
        E2_grid = E2_binary(z_grid, Om)
    elif E2_func is E2_scale_log:
        c = kwargs.get("c", 1.0); E2_grid = E2_scale_log(z_grid, Om, c)
    elif E2_func is E2_offset_log:
        b = kwargs.get("b", 0.0); E2_grid = E2_offset_log(z_grid, Om, b)
    else:
        E2_grid = E2_LCDM(z_grid, Om)

    E_grid = np.sqrt(np.maximum(E2_grid, 1e-30))
    invE = 1.0/E_grid
    Dc_grid = np.zeros_like(z_grid)
    Dc_grid[1:] = np.cumsum(0.5*(invE[1:]+invE[:-1])*(z_grid[1:]-z_grid[:-1]))
    c_km_s = 299792.458
    Dc_grid *= c_km_s/ H0
    Dc = np.interp(z, z_grid, Dc_grid)
    Dl = (1+z)*Dc
    mu = 5.0*np.log10(np.maximum(Dl, 1e-30)) + 25.0
    return mu

class LCDM(CosmoModel):
    def predict_mu(self, z, H0=70.0):
        Om = self.params.get("Omega_m", 0.3)
        return distance_modulus(z, E2_LCDM, H0, Om)

class ConstW(CosmoModel):
    def predict_mu(self, z, H0=70.0):
        Om = self.params.get("Omega_m", 0.3)
        w0 = self.params.get("w0", -1.0)
        return distance_modulus(z, E2_constw, H0, Om, w0=w0)

class W0Wa(CosmoModel):
    def predict_mu(self, z, H0=70.0):
        Om = self.params.get("Omega_m", 0.3)
        w0 = self.params.get("w0", -1.0); wa = self.params.get("wa", 0.0)
        return distance_modulus(z, E2_w0wa, H0, Om, w0=w0, wa=wa)

class BinaryW(CosmoModel):
    def predict_mu(self, z, H0=70.0):
        Om = self.params.get("Omega_m", 0.3)
        return distance_modulus(z, E2_binary, H0, Om)

class ScaleLog(CosmoModel):
    def predict_mu(self, z, H0=70.0):
        Om = self.params.get("Omega_m", 0.3); c = self.params.get("c", 1.0)
        return distance_modulus(z, E2_scale_log, H0, Om, c=c)

class OffsetLog(CosmoModel):
    def predict_mu(self, z, H0=70.0):
        Om = self.params.get("Omega_m", 0.3); b = self.params.get("b", 0.0)
        return distance_modulus(z, E2_offset_log, H0, Om, b=b)

# --------------------------- Data loaders (covariance attempts) ---------------------------

def _try_load_cov_from_text(content: bytes, N: int) -> Optional[np.ndarray]:
    try:
        lines = [ln for ln in content.decode("utf-8", errors="ignore").splitlines() if ln.strip() and not ln.strip().startswith("#")]
        arr = []
        for ln in lines:
            try: arr.append([float(x) for x in ln.split()])
            except Exception: pass
        M = np.array(arr, dtype=float)
        if M.shape == (N, N): return M
        flat = M.reshape(-1); t = flat.size; n = int((np.sqrt(1+8*t)-1)//2)
        if n == N and n*(n+1)//2 == t:
            C = np.zeros((N, N), float); k = 0
            for i in range(N):
                for j in range(i, N):
                    C[i, j] = C[j, i] = flat[k]; k += 1
            return C
        return None
    except Exception:
        return None

def load_union21(use_cov: bool=True) -> Tuple[pd.DataFrame, np.ndarray]:
    tbl = http_get("https://supernova.lbl.gov/Union/figures/SCPUnion2.1_mu_vs_z.txt")
    if tbl is None: raise RuntimeError("Union2.1 table download failed.")
    rows = []
    for ln in tbl.decode("utf-8").splitlines():
        if ln.startswith("#") or not ln.strip(): continue
        p = ln.split()
        if len(p) >= 4: rows.append((float(p[1]), float(p[2]), float(p[3])))
    df = pd.DataFrame(rows, columns=["z","mu","mu_err"]).sort_values("z").reset_index(drop=True)
    N = len(df); C = None
    if use_cov:
        stat = http_get("https://supernova.lbl.gov/Union/figures/SCPUnion2.1_covmat_stat.txt")
        sysm = http_get("https://supernova.lbl.gov/Union/figures/SCPUnion2.1_covmat_sys.txt")
        if stat is not None and sysm is not None:
            C_stat = _try_load_cov_from_text(stat, N); C_sys = _try_load_cov_from_text(sysm, N)
            if C_stat is not None and C_sys is not None: C = C_stat + C_sys
    if C is None: C = np.diag(np.maximum(df["mu_err"].values, 1e-9)**2)
    return df, C

def load_jla(use_cov: bool=True) -> Tuple[pd.DataFrame, np.ndarray]:
    tbl = http_get("http://supernovae.in2p3.fr/sdss_snls_jla/jla_mub.txt")
    if tbl is None: raise RuntimeError("JLA mub download failed.")
    rows = []
    for ln in tbl.decode("utf-8").splitlines():
        if ln.startswith("#") or not ln.strip(): continue
        p = ln.split()
        if len(p) >= 3: rows.append((float(p[0]), float(p[1]), float(p[2])))
    df = pd.DataFrame(rows, columns=["z","mu","mu_err"]).sort_values("z").reset_index(drop=True)
    N = len(df); C = None
    if use_cov:
        for url in [
            "http://supernovae.in2p3.fr/sdss_snls_jla/uncertainties/covmat_v6.dat",
            "http://supernovae.in2p3.fr/sdss_snls_jla/covmat_v6.dat",
            "http://supernovae.in2p3.fr/sdss_snls_jla/jla_cov_full.txt",
        ]:
            blob = http_get(url)
            if blob is not None:
                M = _try_load_cov_from_text(blob, N)
                if M is not None: C = M; break
    if C is None: C = np.diag(np.maximum(df["mu_err"].values, 1e-9)**2)
    return df, C

# --------------------------- Fitting ---------------------------

@dataclass
class FitResult:
    model: str
    dataset: str
    params: Dict[str, float]
    delta: float
    chi2: float
    dof: int
    red_chi2: float
    logL: float
    AIC: float
    BIC: float
    R2: float
    N: int
    tag: str

def fit_model(model: CosmoModel, df: pd.DataFrame, C: np.ndarray, H0: float=70.0) -> FitResult:
    z, mu = df["z"].values, df["mu"].values
    N = len(z)
    jitter = 1e-10 * float(np.mean(np.diag(C)))
    C_use = C + jitter*np.eye(N)
    Cinv = np.linalg.inv(C_use)

    # Param vector (Ωm + model-specific)
    p0, bounds = [], []
    if model.fit_omegam:
        p0.append(model.params.get("Omega_m", 0.3)); bounds.append((0.01, 0.99))
    tag = ""
    if isinstance(model, ConstW):
        p0.append(model.params.get("w0", -1.0)); bounds.append((-3.0, 0.0)); tag="Const‑w"
    if isinstance(model, W0Wa):
        p0.append(model.params.get("w0", -1.0)); bounds.append((-3.0, 0.0))
        p0.append(model.params.get("wa",  0.0)); bounds.append((-3.0, 3.0)); tag="w0–wₐ"
    if isinstance(model, BinaryW):
        tag="Binary"
    if isinstance(model, ScaleLog):
        tag=f"ScaleLog[c={model.params.get('label','?')}]"
    if isinstance(model, OffsetLog):
        tag=f"OffsetLog[b={model.params.get('label','?')}]"

    def make_pred(theta):
        i = 0; pars = dict(model.params)
        if model.fit_omegam: pars["Omega_m"] = float(theta[i]); i += 1
        mdl = type(model)(name=model.name, fit_omegam=model.fit_omegam, params=pars)
        return mdl.predict_mu(z, H0=H0), pars

    def obj(theta):
        mu_pred, _ = make_pred(theta)
        chi2_min, _ = analytic_delta_chi2(mu_obs=mu, mu_model=mu_pred, Cinv=Cinv)
        return chi2_min

    theta = np.array(p0, dtype=float)
    if len(p0) > 0:
        res = minimize(obj, x0=theta, bounds=bounds, method="L-BFGS-B")
        theta = res.x

    mu_pred, pars = make_pred(theta)
    chi2_min, delta = analytic_delta_chi2(mu_obs=mu, mu_model=mu_pred, Cinv=Cinv)
    k = len(theta) + 1  # +Δ
    dof = max(1, N - k)
    logL, AIC, BIC = aic_bic(chi2_min, C_use, k)
    R2 = r_squared(mu, mu_pred + delta)

    return FitResult(
        model=model.name, dataset="", params=pars, delta=float(delta),
        chi2=float(chi2_min), dof=int(dof), red_chi2=float(chi2_min/dof),
        logL=float(logL), AIC=float(AIC), BIC=float(BIC), R2=float(R2), N=N, tag=tag
    )

# --------------------------- Plot & Runner ---------------------------

def panel_plot(df: pd.DataFrame, preds: Dict[str, Dict[str, np.ndarray]], out_png: str, title_suffix=""):
    z, mu, mu_err = df["z"].values, df["mu"].values, df["mu_err"].values
    fig = plt.figure(figsize=(8, 10))

    # Hubble diagram
    ax1 = fig.add_subplot(2, 1, 1)
    ax1.errorbar(z, mu, yerr=mu_err, fmt=".", alpha=0.5, markersize=3)
    for name, d in preds.items():
        ax1.plot(z, d["mu_pred"] + d["delta"])
    ax1.set_xlabel("Redshift z"); ax1.set_ylabel("Distance modulus μ")
    ax1.set_title(f"Supernova Hubble diagram{title_suffix}"); ax1.grid(True, alpha=0.3)
    ax1.legend(list(preds.keys()))

    # Residuals
    ax2 = fig.add_subplot(2, 1, 2)
    for name, d in preds.items():
        res = mu - (d["mu_pred"] + d["delta"])
        ax2.scatter(z, res, s=6, alpha=0.6, label=name)
    ax2.axhline(0.0, linestyle="--")
    ax2.set_xlabel("Redshift z"); ax2.set_ylabel("Residual μ_obs − μ_model")
    ax2.set_title("Residuals (after Δ fit)"); ax2.grid(True, alpha=0.3)
    ax2.legend(list(preds.keys()))

    fig.tight_layout(); fig.savefig(out_png, dpi=200, bbox_inches="tight"); plt.close(fig)

def run_everything(
    outdir="/content/out",
    datasets=("union","jla"),
    use_cov=True,
    include_baselines=True,
    include_binary_original=True,
    include_scale=True,
    include_offset=True,
    H0=70.0
):
    os.makedirs(outdir, exist_ok=True)

    # Load datasets
    ds_list = []
    for token in datasets:
        try:
            if token.lower()=="union": ds_list.append(("Union2.1",)+load_union21(use_cov=use_cov))
            elif token.lower()=="jla": ds_list.append(("JLA",)+load_jla(use_cov=use_cov))
        except Exception as e:
            print(f"[WARN] Failed to load {token}: {e}")
    if not ds_list: raise RuntimeError("No datasets loaded.")

    # Build Fibonacci constants once (transparent printout)
    consts = make_binary_constants(n_digits=200, n_bin_bits=200)
    print("\n=== Binary constants (approx floats) ===")
    for k in ["DF","DR","OneMinusDF","BinF","BinR","neg_DF","neg_DR","neg_OneMinusDF","neg_BinF","neg_BinR"]:
        print(f"{k:>14} = {consts[k]:.18g}")

    # Prepare model list
    models: List[CosmoModel] = []
    if include_baselines:
        models += [
            LCDM(name="ΛCDM", fit_omegam=True, params={"Omega_m":0.3}),
            ConstW(name="Const‑w", fit_omegam=True, params={"Omega_m":0.3,"w0":-1.0}),
            W0Wa(name="w0–wₐ", fit_omegam=True, params={"Omega_m":0.3,"w0":-1.0,"wa":0.0}),
        ]
    if include_binary_original:
        models.append(BinaryW(name="Binary w(a)=−log₂a", fit_omegam=True, params={"Omega_m":0.3}))
    if include_scale:
        for key in ["DF","DR","OneMinusDF","BinF","BinR","neg_DF","neg_DR","neg_OneMinusDF","neg_BinF","neg_BinR"]:
            val = consts[key]
            models.append(ScaleLog(name=f"ScaleLog c≈{val:.6g}", fit_omegam=True,
                                   params={"Omega_m":0.3, "c": val, "label": key}))
    if include_offset:
        for key in ["DF","DR","OneMinusDF","BinF","BinR","neg_DF","neg_DR","neg_OneMinusDF","neg_BinF","neg_BinR"]:
            val = consts[key]
            models.append(OffsetLog(name=f"OffsetLog b≈{val:.6g}", fit_omegam=True,
                                    params={"Omega_m":0.3, "b": val, "label": key}))

    # Fit on each dataset
    rows = []
    for (ds_name, df, C) in ds_list:
        print(f"\n=== DATASET: {ds_name} (N={len(df)}) ===")
        z = df["z"].values
        preds_for_plot = {}
        # We will display: the three baselines + best offset‑log + best scale‑log by AIC
        aic_cache = []

        for mdl in models:
            print(f"Fitting {mdl.name} ...")
            res = fit_model(mdl, df, C, H0=H0); res.dataset = ds_name
            row = {
                "dataset": ds_name,
                "model": mdl.name,
                **{f"param_{k}": v for k, v in res.params.items()},
                "delta": res.delta,
                "chi2": res.chi2, "dof": res.dof, "red_chi2": res.red_chi2,
                "logL": res.logL, "AIC": res.AIC, "BIC": res.BIC, "R2": res.R2, "N": res.N, "tag": res.tag
            }
            rows.append(row)
            aic_cache.append((res.AIC, mdl, res))

        # pick top ScaleLog and OffsetLog by AIC to overlay with baselines
        df_tmp = pd.DataFrame(rows)
        df_ds = df_tmp[df_tmp["dataset"]==ds_name]
        def pick_best(prefix):
            sub = df_ds[df_ds["tag"].str.startswith(prefix)]
            return None if sub.empty else sub.sort_values("AIC").iloc[0]
        best_scale  = pick_best("ScaleLog")
        best_offset = pick_best("OffsetLog")

        # always plot ΛCDM, Const‑w, original Binary; add the best of each family
        lines = []
        for label in ["ΛCDM","Const‑w","Binary w(a)=−log₂a"]:
            hit = df_ds[df_ds["model"]==label]
            if not hit.empty: lines.append(hit.iloc[0])
        for extra in [best_scale, best_offset]:
            if extra is not None: lines.append(extra)

        # build predictions for plot
        for r in lines:
            # rebuild model to generate μ(z)
            if r["tag"].startswith("ScaleLog"):
                mdl = ScaleLog(name=r["model"], fit_omegam=True, params={"Omega_m":r.get("param_Omega_m",0.3), "c": r.get("param_c", consts.get("DF",1.0))})
                mu_pred = mdl.predict_mu(z)
            elif r["tag"].startswith("OffsetLog"):
                mdl = OffsetLog(name=r["model"], fit_omegam=True, params={"Omega_m":r.get("param_Omega_m",0.3), "b": r.get("param_b", consts.get("DF",0.0))})
                mu_pred = mdl.predict_mu(z)
            elif r["model"]=="ΛCDM":
                mdl = LCDM(name=r["model"], fit_omegam=True, params={"Omega_m":r.get("param_Omega_m",0.3)}); mu_pred = mdl.predict_mu(z)
            elif r["model"]=="Const‑w":
                mdl = ConstW(name=r["model"], fit_omegam=True, params={"Omega_m":r.get("param_Omega_m",0.3), "w0": r.get("param_w0",-1.0)}); mu_pred = mdl.predict_mu(z)
            elif r["model"]=="Binary w(a)=−log₂a":
                mdl = BinaryW(name=r["model"], fit_omegam=True, params={"Omega_m":r.get("param_Omega_m",0.3)}); mu_pred = mdl.predict_mu(z)
            else:
                continue
            preds_for_plot[r["model"]] = {"mu_pred": mu_pred, "delta": r["delta"]}

        out_png = os.path.join(outdir, f"hubble_{ds_name.replace(' ','_')}.png")
        panel_plot(df, preds_for_plot, out_png, title_suffix=f" — {ds_name}")
        print(f"Saved figure: {out_png}")

    # Save results with ΔAIC/ΔBIC
    df_res = pd.DataFrame(rows)
    df_res["rank_by_AIC"] = df_res.groupby("dataset")["AIC"].rank(method="min")
    df_res["delta_AIC"]  = df_res["AIC"] - df_res.groupby("dataset")["AIC"].transform("min")
    df_res["rank_by_BIC"] = df_res.groupby("dataset")["BIC"].rank(method="min")
    df_res["delta_BIC"]  = df_res["BIC"] - df_res.groupby("dataset")["BIC"].transform("min")
    out_csv = os.path.join(outdir, "results_summary.csv")
    df_res.to_csv(out_csv, index=False)
    print(f"\nWrote results: {out_csv}")
    print("\nPrimary verdict rule: rank by reduced χ² and ΔAIC (≤ 2 ≈ indistinguishable). R² is descriptive only.")
    show_cols = ["dataset","model","tag","red_chi2","AIC","delta_AIC","BIC","delta_BIC"]
    try:
        from IPython.display import display
        display(df_res[show_cols].sort_values(["dataset","AIC"]).head(20))
    except Exception:
        print(df_res[show_cols].sort_values(["dataset","AIC"]).head(20))

# --------------------------- Run with robust defaults ---------------------------
run_everything(
    outdir="/content/out",
    datasets=("union","jla"),     # JLA sometimes times out; Union2.1 will still run
    use_cov=True,                 # try covariance, fall back to diagonal automatically
    include_baselines=True,
    include_binary_original=True,
    include_scale=True,
    include_offset=True,
    H0=70.0
)


[WARN] Failed to load jla: JLA mub download failed.

=== Binary constants (approx floats) ===
            DF = 0.0100101001001010013
            DR = 0.101101011011010114
    OneMinusDF = 0.989989899899899051
          BinF = 0.290196557138708677
          BinR = 0.709803442861291378
        neg_DF = -0.0100101001001010013
        neg_DR = -0.101101011011010114
neg_OneMinusDF = -0.989989899899899051
      neg_BinF = -0.290196557138708677
      neg_BinR = -0.709803442861291378

=== DATASET: Union2.1 (N=580) ===
Fitting ΛCDM ...
Fitting Const‑w ...
Fitting w0–wₐ ...
Fitting Binary w(a)=−log₂a ...
Fitting ScaleLog c≈0.0100101 ...
Fitting ScaleLog c≈0.101101 ...
Fitting ScaleLog c≈0.98999 ...
Fitting ScaleLog c≈0.290197 ...
Fitting ScaleLog c≈0.709803 ...
Fitting ScaleLog c≈-0.0100101 ...
Fitting ScaleLog c≈-0.101101 ...
Fitting ScaleLog c≈-0.98999 ...
Fitting ScaleLog c≈-0.290197 ...
Fitting ScaleLog c≈-0.709803 ...
Fitting OffsetLog b≈0.0100101 ...
Fitting OffsetLog b≈0.101101 ...
Fittin

,dataset,model,tag,red_chi2,AIC,delta_AIC,BIC,delta_BIC
0,Union2.1,ΛCDM,,0.972710,-233.480557,0.000000,-224.754501,0.000000
1,Union2.1,Const‑w,Const‑w,0.974396,-231.480557,2.000000,-218.391473,6.363028
2,Union2.1,w0–wₐ,w0–wₐ,0.976088,-229.480557,4.000000,-212.028445,12.726056
16,Union2.1,OffsetLog b≈0.98999,OffsetLog[b=OneMinusDF],0.979926,-229.310078,4.170480,-220.584021,4.170480
18,Union2.1,OffsetLog b≈0.709803,OffsetLog[b=BinR],1.084445,-168.898089,64.582469,-160.172032,64.582469
11,Union2.1,ScaleLog c≈-0.98999,ScaleLog[c=neg_OneMinusDF],1.304736,-41.569659,191.910899,-32.843602,191.910899
13,Union2.1,ScaleLog c≈-0.709803,ScaleLog[c=neg_BinR],1.450802,42.856384,276.336942,51.582441,276.336942
12,Union2.1,ScaleLog c≈-0.290197,ScaleLog[c=neg_BinF],1.759130,221.069724,454.550281,229.795780,454.550281
10,Union2.1,ScaleLog c≈-0.101101,ScaleLog[c=neg_DR],1.928869,319.179078,552.659635,327.905134,552.659635
9,Union2.1,ScaleLog c≈-0.0100101,ScaleLog[c=neg_DF],2.016748,369.973123,603.453680,378.699179,603.453680
